In [12]:
%load_ext watermark
%watermark -p torch,lightning,torchvision

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
torch      : 2.7.0+cu128
lightning  : 2.5.2
torchvision: 0.22.0+cu128



In [13]:

import lightning as L
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torchmetrics
from lightning.pytorch.loggers import CSVLogger
import matplotlib.pyplot as plt
import numpy as np

from shared_utilities import LightningModel,Cifar10DataModule,plot_loss_and_acc

In [22]:
# Loading up model and matching the FC layer to the pretrained model architecture

pytorch_model = torch.hub.load('pytorch/vision', 'resnet18', weights=None)
pytorch_model.fc = torch.nn.Sequential(
    nn.Linear(512,512),
    nn.ReLU(),
    nn.Linear(512,216) # We will get the embeddings !! not not classes to further finetune for specific class!! 
)

pytorch_model.load_state_dict(torch.load('simclr-resnet18.pt'))
pytorch_model.fc = torch.nn.Linear(512, 10)

Using cache found in /home/zeus/.cache/torch/hub/pytorch_vision_main


In [ ]:
%%capture --no-display

L.seed_everything(123)
dm = Cifar10DataModule(height_width=(128, 128), batch_size=64, num_workers=4)

lightning_model = LightningModel(model=pytorch_model, learning_rate=0.1)

trainer = L.Trainer(
    max_epochs=50,
    accelerator="gpu",
    devices=[1],
    logger=CSVLogger(save_dir="logs/", name="my-model"),
    deterministic=True,
)

In [28]:

trainer.fit(model=lightning_model, datamodule=dm)


  | Name      | Type               | Params | Mode 
---------------------------------------------------------
0 | model     | ResNet             | 11.5 M | train
1 | train_acc | MulticlassAccuracy | 0      | train
2 | val_acc   | MulticlassAccuracy | 0      | train
3 | test_acc  | MulticlassAccuracy | 0      | train
---------------------------------------------------------
11.5 M    Trainable params
0         Non-trainable params
11.5 M    Total params
46.200    Total estimated model params size (MB)
74        Modules in train mode
0         Modules in eval mode


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_steps=1` reached.


In [ ]:

plot_loss_and_acc(trainer.logger.log_dir)

In [ ]:
trainer.test(model=lightning_model, datamodule=dm)